# Week 9: Limits and Continuity - Practice Notebook---**Date**: 2025-11-21  **Course**: BSMA1001 - Mathematics for Data Science I  **Level**: Foundation  **Week**: 9 of 12  **Topic Area**: Calculus - Limits and Continuity---## Learning ObjectivesThis notebook provides hands-on practice with:- Visualizing limits and their ε-δ definitions- Computing limits algebraically- Evaluating one-sided and infinite limits- Testing continuity at points- Identifying and classifying discontinuities- Applying the Intermediate Value Theorem- Using limits in optimization and root finding## Prerequisites```pythonimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom scipy.optimize import fsolve, brentq```

In [ ]:
import numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom scipy.optimize import fsolve, brentqimport warningswarnings.filterwarnings('ignore')# Set random seednp.random.seed(42)# Configure plottingplt.style.use('seaborn-v0_8-darkgrid')sns.set_palette("husl")print("✓ Libraries imported successfully")print(f"NumPy version: {np.__version__}")

## 1. Visualizing LimitsLet's start by visualizing what it means for a function to approach a limit.**Example:** $f(x) = \frac{x^2 - 4}{x - 2}$ as $x \to 2$

In [ ]:
def f_removable(x):    """Function with removable discontinuity at x=2."""    return (x**2 - 4) / (x - 2)def f_simplified(x):    """Simplified form: x + 2"""    return x + 2# Create x values avoiding x=2x_vals = np.linspace(-1, 5, 1000)x_vals = x_vals[np.abs(x_vals - 2) > 0.01]  # Exclude points very close to 2# Calculate y valuesy_vals = f_removable(x_vals)# Create figurefig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))# Left plot: Show the function with holeax1.plot(x_vals, y_vals, 'b-', linewidth=2, label='$f(x) = \\frac{x^2-4}{x-2}$')ax1.plot(2, 4, 'ro', markersize=10, fillstyle='none', markeredgewidth=2, label='Hole at x=2')ax1.axhline(y=4, color='r', linestyle='--', alpha=0.5, linewidth=1)ax1.axvline(x=2, color='r', linestyle='--', alpha=0.5, linewidth=1)ax1.set_xlabel('x', fontsize=12)ax1.set_ylabel('f(x)', fontsize=12)ax1.set_title('Function with Removable Discontinuity', fontsize=13, fontweight='bold')ax1.legend(fontsize=10)ax1.grid(True, alpha=0.3)ax1.set_ylim(-1, 8)ax1.text(2, 4.5, 'Limit = 4', fontsize=11, ha='center')# Right plot: Zoom in near x=2x_close = np.linspace(1.5, 2.5, 100)x_close = x_close[np.abs(x_close - 2) > 0.001]y_close = f_removable(x_close)ax2.plot(x_close, y_close, 'b-', linewidth=2)ax2.plot(2, 4, 'ro', markersize=12, fillstyle='none', markeredgewidth=2)ax2.axhline(y=4, color='r', linestyle='--', alpha=0.5, linewidth=2)ax2.axvline(x=2, color='r', linestyle='--', alpha=0.5, linewidth=2)ax2.fill_between([1.5, 2.5], 3.8, 4.2, alpha=0.2, color='green', label='ε-neighborhood')ax2.set_xlabel('x', fontsize=12)ax2.set_ylabel('f(x)', fontsize=12)ax2.set_title('Zoomed View Near x=2', fontsize=13, fontweight='bold')ax2.legend(fontsize=10)ax2.grid(True, alpha=0.3)plt.tight_layout()plt.show()# Compute values approaching 2print("Values as x approaches 2:")print("=" * 50)for x in [1.9, 1.99, 1.999, 2.001, 2.01, 2.1]:    if abs(x - 2) > 1e-10:        y = f_removable(x)        print(f"x = {x:6.3f}  →  f(x) = {y:.6f}")    print(f"\nLimit as x → 2: {f_simplified(2)}")

## 2. Epsilon-Delta (ε-δ) Definition**Formal definition:** $\lim_{x \to a} f(x) = L$ means:For every $\varepsilon > 0$, there exists $\delta > 0$ such that:$$0 < |x - a| < \delta \implies |f(x) - L| < \varepsilon$$Let's visualize this for $f(x) = 2x + 1$ as $x \to 3$ (limit = 7).

In [ ]:
def epsilon_delta_viz(f, a, L, epsilon_vals):    """Visualize epsilon-delta definition."""    fig, axes = plt.subplots(1, len(epsilon_vals), figsize=(5*len(epsilon_vals), 5))        if len(epsilon_vals) == 1:        axes = [axes]        x = np.linspace(a - 2, a + 2, 1000)    y = f(x)        for idx, eps in enumerate(epsilon_vals):        ax = axes[idx]                # Plot function        ax.plot(x, y, 'b-', linewidth=2, label='f(x) = 2x + 1')                # Epsilon neighborhood around L        ax.fill_between([a-2, a+2], L-eps, L+eps, alpha=0.2, color='green',                         label=f'ε = {eps} neighborhood')        ax.axhline(y=L+eps, color='green', linestyle='--', linewidth=1)        ax.axhline(y=L-eps, color='green', linestyle='--', linewidth=1)        ax.axhline(y=L, color='r', linestyle='-', linewidth=2, alpha=0.7)                # Find delta        # For f(x) = 2x + 1, |f(x) - 7| < eps means |2x - 6| < eps        # So |x - 3| < eps/2, thus delta = eps/2        delta = eps / 2                # Delta neighborhood around a        ax.axvline(x=a, color='r', linestyle='-', linewidth=2, alpha=0.7)        ax.axvline(x=a-delta, color='orange', linestyle='--', linewidth=1.5)        ax.axvline(x=a+delta, color='orange', linestyle='--', linewidth=1.5)        ax.fill_betweenx([y.min(), y.max()], a-delta, a+delta, alpha=0.15,                          color='orange', label=f'δ = {delta:.2f} neighborhood')                ax.set_xlabel('x', fontsize=12)        ax.set_ylabel('f(x)', fontsize=12)        ax.set_title(f'ε = {eps}, δ = {delta:.2f}', fontsize=13, fontweight='bold')        ax.legend(fontsize=9, loc='upper left')        ax.grid(True, alpha=0.3)        ax.set_xlim(a-1.5, a+1.5)        ax.set_ylim(L-3, L+3)        plt.tight_layout()    plt.show()# Test with f(x) = 2x + 1, a = 3, L = 7f_linear = lambda x: 2*x + 1epsilon_delta_viz(f_linear, a=3, L=7, epsilon_vals=[0.5, 1.0, 2.0])print("Observation: Smaller ε requires smaller δ")print("For f(x) = 2x + 1: δ = ε/2 works!")

## 3. Computing Limits Algebraically### Method 1: Direct Substitution (if no indeterminate form)### Method 2: Factoring and Canceling (for 0/0 forms)### Method 3: Rationalization (for radicals)Let's practice each technique:

In [ ]:
def compute_limit_numerical(f, a, side='both'):    """Numerically approximate limit."""    h_vals = [0.1, 0.01, 0.001, 0.0001]        if side in ['both', 'left']:        print(f"Approaching from left (x → {a}⁻):")        for h in h_vals:            try:                val = f(a - h)                print(f"  x = {a - h:.4f}: f(x) = {val:.8f}")            except:                print(f"  x = {a - h:.4f}: undefined")        if side in ['both', 'right']:        print(f"\nApproaching from right (x → {a}⁺):")        for h in h_vals:            try:                val = f(a + h)                print(f"  x = {a + h:.4f}: f(x) = {val:.8f}")            except:                print(f"  x = {a + h:.4f}: undefined")# Example 1: Factoringprint("Example 1: lim(x→3) (x² - 9)/(x - 3)")print("=" * 60)f1 = lambda x: (x**2 - 9) / (x - 3)compute_limit_numerical(f1, 3)print(f"\nAlgebraic solution: (x² - 9)/(x - 3) = (x-3)(x+3)/(x-3) = x + 3")print(f"Limit = 3 + 3 = 6")print("\n" + "=" * 60)print("\nExample 2: lim(x→0) (√(x+4) - 2)/x")print("=" * 60)f2 = lambda x: (np.sqrt(x + 4) - 2) / xcompute_limit_numerical(f2, 0)print(f"\nAlgebraic solution: Multiply by conjugate")print(f"= (x+4-4) / [x(√(x+4)+2)] = x / [x(√(x+4)+2)]")print(f"= 1 / (√(x+4)+2) → 1/4 as x → 0")print(f"Limit = 0.25")

## 4. One-Sided Limits**Left-hand limit:** $\lim_{x \to a^-} f(x)$ (approach from left)**Right-hand limit:** $\lim_{x \to a^+} f(x)$ (approach from right)**Theorem:** $\lim_{x \to a} f(x)$ exists ⟺ both one-sided limits exist and are equal.

In [ ]:
def plot_piecewise(x_ranges, funcs, a, title):    """Plot piecewise function and show one-sided limits."""    fig, ax = plt.subplots(figsize=(10, 6))        colors = ['blue', 'red', 'green', 'purple']        for idx, (x_range, func) in enumerate(zip(x_ranges, funcs)):        x = np.linspace(x_range[0], x_range[1], 500)        y = func(x)        ax.plot(x, y, color=colors[idx], linewidth=2.5)        # Evaluate one-sided limits    try:        left_limit = funcs[0](a - 0.001)        ax.plot(a, left_limit, 'o', color='blue', markersize=10,                 fillstyle='none', markeredgewidth=2, label=f'Left limit = {left_limit:.2f}')    except:        pass        try:        right_limit = funcs[-1](a + 0.001)        ax.plot(a, right_limit, 's', color='red', markersize=10,                fillstyle='none', markeredgewidth=2, label=f'Right limit = {right_limit:.2f}')    except:        pass        # Try to find actual function value    for func in funcs:        try:            val = func(a)            ax.plot(a, val, 'ko', markersize=8, label=f'f({a}) = {val:.2f}')            break        except:            pass        ax.axvline(x=a, color='gray', linestyle='--', alpha=0.5, linewidth=1.5)    ax.set_xlabel('x', fontsize=12)    ax.set_ylabel('f(x)', fontsize=12)    ax.set_title(title, fontsize=14, fontweight='bold')    ax.legend(fontsize=11)    ax.grid(True, alpha=0.3)    plt.show()# Example 1: Jump discontinuityprint("Example 1: Piecewise function with jump")print("f(x) = { x² if x < 1, 2x if x ≥ 1 }")f_left = lambda x: x**2f_right = lambda x: 2*xplot_piecewise(    x_ranges=[(-1, 1), (1, 3)],    funcs=[f_left, f_right],    a=1,    title='Jump Discontinuity at x=1')print("Left limit: lim(x→1⁻) x² = 1")print("Right limit: lim(x→1⁺) 2x = 2")print("Limit does NOT exist (1 ≠ 2)")# Example 2: Absolute value divided by xprint("\n" + "="*60)print("\nExample 2: f(x) = |x|/x")f_abs_left = lambda x: -1 * np.ones_like(x)f_abs_right = lambda x: 1 * np.ones_like(x)plot_piecewise(    x_ranges=[(-2, 0), (0, 2)],    funcs=[f_abs_left, f_abs_right],    a=0,    title='f(x) = |x|/x at x=0')print("Left limit: lim(x→0⁻) |x|/x = lim(x→0⁻) -x/x = -1")print("Right limit: lim(x→0⁺) |x|/x = lim(x→0⁺) x/x = 1")print("Limit does NOT exist (-1 ≠ 1)")print("\nML Connection: ReLU activation has different derivatives from left/right at 0")

## 5. Infinite Limits and Vertical Asymptotes**Notation:** $\lim_{x \to a} f(x) = \infty$ (or $-\infty$)**Vertical asymptote** at $x = a$ if function approaches $\pm\infty$ as $x \to a$.

In [ ]:
def plot_vertical_asymptote(f, asymptote, x_range, title):    """Plot function with vertical asymptote."""    fig, ax = plt.subplots(figsize=(10, 6))        # Create x values avoiding asymptote    x1 = np.linspace(x_range[0], asymptote - 0.05, 200)    x2 = np.linspace(asymptote + 0.05, x_range[1], 200)        y1 = f(x1)    y2 = f(x2)        # Plot both sides    ax.plot(x1, y1, 'b-', linewidth=2, label='f(x)')    ax.plot(x2, y2, 'b-', linewidth=2)        # Vertical asymptote    ax.axvline(x=asymptote, color='r', linestyle='--', linewidth=2,                label=f'Vertical asymptote: x={asymptote}')        ax.set_xlabel('x', fontsize=12)    ax.set_ylabel('f(x)', fontsize=12)    ax.set_title(title, fontsize=14, fontweight='bold')    ax.set_ylim(-10, 10)    ax.legend(fontsize=11)    ax.grid(True, alpha=0.3)    plt.show()# Example 1: 1/(x-2)print("Example 1: f(x) = 1/(x-2)")f1 = lambda x: 1 / (x - 2)plot_vertical_asymptote(f1, asymptote=2, x_range=(0, 4),                         title='f(x) = 1/(x-2) with Vertical Asymptote at x=2')print("lim(x→2⁺) 1/(x-2) = +∞")print("lim(x→2⁻) 1/(x-2) = -∞")# Example 2: 1/x²print("\n" + "="*60)print("\nExample 2: f(x) = 1/x²")f2 = lambda x: 1 / x**2plot_vertical_asymptote(f2, asymptote=0, x_range=(-2, 2),                        title='f(x) = 1/x² with Vertical Asymptote at x=0')print("lim(x→0⁺) 1/x² = +∞")print("lim(x→0⁻) 1/x² = +∞")print("(Both sides approach +∞)")# Multiple asymptotesprint("\n" + "="*60)print("\nExample 3: f(x) = 1/((x-1)(x+1))")f3 = lambda x: 1 / ((x - 1) * (x + 1))fig, ax = plt.subplots(figsize=(12, 6))# Plot in segments avoiding asymptotessegments = [    np.linspace(-3, -1.1, 200),    np.linspace(-0.9, 0.9, 200),    np.linspace(1.1, 3, 200)]for seg in segments:    ax.plot(seg, f3(seg), 'b-', linewidth=2)ax.axvline(x=-1, color='r', linestyle='--', linewidth=2, label='Asymptotes')ax.axvline(x=1, color='r', linestyle='--', linewidth=2)ax.set_xlabel('x', fontsize=12)ax.set_ylabel('f(x)', fontsize=12)ax.set_title('Multiple Vertical Asymptotes at x=-1 and x=1', fontsize=14, fontweight='bold')ax.set_ylim(-10, 10)ax.legend(fontsize=11)ax.grid(True, alpha=0.3)plt.show()

## 6. Limits at Infinity and Horizontal Asymptotes**Horizontal asymptote** at $y = L$ if:- $\lim_{x \to \infty} f(x) = L$, or- $\lim_{x \to -\infty} f(x) = L$**Technique for rational functions:** Divide by highest power in denominator.

In [ ]:
def plot_horizontal_asymptote(f, asymptote, x_range, title):    """Plot function with horizontal asymptote."""    fig, ax = plt.subplots(figsize=(12, 6))        x = np.linspace(x_range[0], x_range[1], 1000)    y = f(x)        ax.plot(x, y, 'b-', linewidth=2, label='f(x)')        if asymptote is not None:        ax.axhline(y=asymptote, color='r', linestyle='--', linewidth=2,                   label=f'Horizontal asymptote: y={asymptote}')        ax.set_xlabel('x', fontsize=12)    ax.set_ylabel('f(x)', fontsize=12)    ax.set_title(title, fontsize=14, fontweight='bold')    ax.legend(fontsize=11)    ax.grid(True, alpha=0.3)    plt.show()# Example 1: Same degree numerator/denominatorprint("Example 1: lim(x→∞) (3x² + 2x - 1)/(x² + 5)")f1 = lambda x: (3*x**2 + 2*x - 1) / (x**2 + 5)plot_horizontal_asymptote(f1, asymptote=3, x_range=(-10, 10),                          title='f(x) = (3x² + 2x - 1)/(x² + 5)')print("Divide by x²:")print("  Numerator: 3 + 2/x - 1/x²")print("  Denominator: 1 + 5/x²")print("As x → ∞: terms with x in denominator → 0")print("Limit = 3/1 = 3")# Example 2: Numerator degree < denominator degreeprint("\n" + "="*60)print("\nExample 2: lim(x→∞) (2x + 1)/(x² - 4)")f2 = lambda x: (2*x + 1) / (x**2 - 4)plot_horizontal_asymptote(f2, asymptote=0, x_range=(-10, 10),                          title='f(x) = (2x + 1)/(x² - 4)')print("Divide by x²:")print("  Numerator: 2/x + 1/x²")print("  Denominator: 1 - 4/x²")print("As x → ∞: (2/x + 1/x²) → 0")print("Limit = 0/1 = 0")# Example 3: Numerator degree > denominator degreeprint("\n" + "="*60)print("\nExample 3: lim(x→∞) (x³ - 2)/(x² + 1)")f3 = lambda x: (x**3 - 2) / (x**2 + 1)x_plot = np.linspace(-5, 5, 1000)y_plot = f3(x_plot)fig, ax = plt.subplots(figsize=(12, 6))ax.plot(x_plot, y_plot, 'b-', linewidth=2)ax.set_xlabel('x', fontsize=12)ax.set_ylabel('f(x)', fontsize=12)ax.set_title('f(x) = (x³ - 2)/(x² + 1) - No Horizontal Asymptote', fontsize=14, fontweight='bold')ax.set_ylim(-50, 50)ax.grid(True, alpha=0.3)plt.show()print("Divide by x²:")print("  Numerator: x - 2/x²")print("  Denominator: 1 + 1/x²")print("As x → ∞: x → ∞")print("Limit = ∞ (no horizontal asymptote)")

## 7. Testing ContinuityA function $f$ is **continuous at $x = a$** if:1. $f(a)$ is defined2. $\lim_{x \to a} f(x)$ exists3. $\lim_{x \to a} f(x) = f(a)$Let's create a function to test continuity systematically:

In [ ]:
def test_continuity(f, a, f_a=None, epsilon=1e-6):    """Test if function is continuous at point a."""    print(f"Testing continuity at x = {a}")    print("=" * 60)        # Test 1: Is f(a) defined?    if f_a is None:        try:            f_a = f(a)            print(f"✓ Condition 1: f({a}) = {f_a:.6f} (defined)")            cond1 = True        except:            print(f"✗ Condition 1: f({a}) is undefined")            return False    else:        print(f"✓ Condition 1: f({a}) = {f_a:.6f} (given)")        cond1 = True        # Test 2: Does limit exist?    try:        # Left limit        left_vals = [f(a - h) for h in [0.1, 0.01, 0.001]]        left_limit = left_vals[-1]                # Right limit        right_vals = [f(a + h) for h in [0.1, 0.01, 0.001]]        right_limit = right_vals[-1]                print(f"  Left limit:  {left_limit:.6f}")        print(f"  Right limit: {right_limit:.6f}")                if abs(left_limit - right_limit) < epsilon:            limit = (left_limit + right_limit) / 2            print(f"✓ Condition 2: lim(x→{a}) f(x) = {limit:.6f} (exists)")            cond2 = True        else:            print(f"✗ Condition 2: Left and right limits differ")            return False    except:        print(f"✗ Condition 2: Limit does not exist or cannot be computed")        return False        # Test 3: Does limit equal f(a)?    if abs(limit - f_a) < epsilon:        print(f"✓ Condition 3: lim(x→{a}) f(x) = f({a}) = {f_a:.6f}")        print(f"\n✓✓✓ Function IS CONTINUOUS at x = {a}")        return True    else:        print(f"✗ Condition 3: lim(x→{a}) f(x) = {limit:.6f} ≠ f({a}) = {f_a:.6f}")        print(f"\n→ Removable discontinuity")        return False# Test examplesprint("Example 1: f(x) = x² - 3x + 2 at x = 2")f1 = lambda x: x**2 - 3*x + 2test_continuity(f1, 2)print("\n" + "="*60)print("\nExample 2: f(x) = (x² - 9)/(x - 3) at x = 3")f2 = lambda x: (x**2 - 9) / (x - 3)test_continuity(f2, 3)print("\n" + "="*60)print("\nExample 3: Piecewise with f(1) = 1, but lim = 2")def f3(x):    if x < 1:        return x**2    else:        return 2*xtest_continuity(f3, 1, f_a=1)  # Manually specify f(1) = 1

## 8. Types of Discontinuities1. **Removable**: Limit exists but ≠ f(a) (or f(a) undefined)2. **Jump**: Left and right limits exist but differ3. **Infinite**: Function approaches ±∞4. **Oscillating**: Function oscillates infinitelyLet's visualize each type:

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))axes = axes.flatten()# 1. Removable discontinuityax = axes[0]x1 = np.linspace(-2, 1.95, 200)x2 = np.linspace(2.05, 4, 200)y1 = (x1**2 - 4) / (x1 - 2)y2 = (x2**2 - 4) / (x2 - 2)ax.plot(x1, y1, 'b-', linewidth=2)ax.plot(x2, y2, 'b-', linewidth=2)ax.plot(2, 4, 'ro', markersize=10, fillstyle='none', markeredgewidth=2, label='Hole')ax.plot(2, 1, 'ko', markersize=8, label='f(2) = 1')ax.axhline(y=4, color='g', linestyle='--', alpha=0.5, linewidth=1)ax.set_title('Removable Discontinuity', fontsize=13, fontweight='bold')ax.set_xlabel('x', fontsize=11)ax.set_ylabel('f(x)', fontsize=11)ax.legend(fontsize=10)ax.grid(True, alpha=0.3)# 2. Jump discontinuityax = axes[1]x1 = np.linspace(-2, 1, 200)x2 = np.linspace(1, 4, 200)y1 = x1**2y2 = 2*x2ax.plot(x1, y1, 'b-', linewidth=2, label='x < 1: x²')ax.plot(x2, y2, 'r-', linewidth=2, label='x ≥ 1: 2x')ax.plot(1, 1, 'bo', markersize=10, fillstyle='none', markeredgewidth=2)ax.plot(1, 2, 'ro', markersize=8)ax.axvline(x=1, color='gray', linestyle='--', alpha=0.5)ax.set_title('Jump Discontinuity', fontsize=13, fontweight='bold')ax.set_xlabel('x', fontsize=11)ax.set_ylabel('f(x)', fontsize=11)ax.legend(fontsize=10)ax.grid(True, alpha=0.3)# 3. Infinite discontinuityax = axes[2]x1 = np.linspace(-2, -0.05, 200)x2 = np.linspace(0.05, 2, 200)y1 = 1 / x1y2 = 1 / x2ax.plot(x1, y1, 'b-', linewidth=2)ax.plot(x2, y2, 'b-', linewidth=2)ax.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Vertical asymptote')ax.set_title('Infinite Discontinuity', fontsize=13, fontweight='bold')ax.set_xlabel('x', fontsize=11)ax.set_ylabel('f(x)', fontsize=11)ax.set_ylim(-10, 10)ax.legend(fontsize=10)ax.grid(True, alpha=0.3)# 4. Oscillating discontinuityax = axes[3]x1 = np.linspace(-0.5, -0.001, 1000)x2 = np.linspace(0.001, 0.5, 1000)y1 = np.sin(1/x1)y2 = np.sin(1/x2)ax.plot(x1, y1, 'b-', linewidth=1, alpha=0.7)ax.plot(x2, y2, 'b-', linewidth=1, alpha=0.7)ax.axvline(x=0, color='r', linestyle='--', linewidth=2, label='x=0')ax.fill_between([-0.5, 0.5], -1, 1, alpha=0.15, color='orange')ax.set_title('Oscillating Discontinuity: sin(1/x)', fontsize=13, fontweight='bold')ax.set_xlabel('x', fontsize=11)ax.set_ylabel('f(x)', fontsize=11)ax.set_ylim(-1.5, 1.5)ax.legend(fontsize=10)ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()print("Discontinuity Classification:")print("  1. Removable: Can be 'fixed' by redefining function value")print("  2. Jump: Different limits from left/right (step functions)")print("  3. Infinite: Vertical asymptotes")print("  4. Oscillating: Rapid oscillations prevent limit from existing")

## 9. Intermediate Value Theorem (IVT)**Theorem:** If $f$ is continuous on $[a, b]$ and $k$ is between $f(a)$ and $f(b)$, then there exists $c \in (a, b)$ such that $f(c) = k$.**Application:** Proving existence of roots (zeros) of equations.

In [ ]:
def ivt_root_finder(f, a, b, visualize=True):    """Use IVT to find root in [a, b]."""    fa, fb = f(a), f(b)        print(f"Finding root of f(x) = 0 in [{a}, {b}]")    print("=" * 60)    print(f"f({a}) = {fa:.6f}")    print(f"f({b}) = {fb:.6f}")        # Check if IVT applies    if fa * fb > 0:        print("\n✗ IVT does not apply: f(a) and f(b) have same sign")        return None        print("\n✓ IVT applies: opposite signs guarantee root exists")        # Use bisection method    root = brentq(f, a, b)    print(f"\nRoot found at x = {root:.8f}")    print(f"Verification: f({root:.8f}) = {f(root):.2e}")        if visualize:        x = np.linspace(a, b, 1000)        y = f(x)                fig, ax = plt.subplots(figsize=(10, 6))        ax.plot(x, y, 'b-', linewidth=2, label='f(x)')        ax.axhline(y=0, color='k', linestyle='-', linewidth=1, alpha=0.3)        ax.plot([a, b], [fa, fb], 'ro', markersize=10, label=f'Endpoints')        ax.plot(root, 0, 'go', markersize=12, label=f'Root ≈ {root:.4f}')        ax.fill_between([a, b], min(fa, fb, 0), max(fa, fb, 0), alpha=0.2, color='green')        ax.set_xlabel('x', fontsize=12)        ax.set_ylabel('f(x)', fontsize=12)        ax.set_title('Intermediate Value Theorem: Root Finding', fontsize=14, fontweight='bold')        ax.legend(fontsize=11)        ax.grid(True, alpha=0.3)        plt.show()        return root# Example 1: x³ - 2x - 5 = 0print("Example 1: x³ - 2x - 5 = 0 in [2, 3]")f1 = lambda x: x**3 - 2*x - 5root1 = ivt_root_finder(f1, 2, 3)# Example 2: cos(x) = xprint("\n" + "="*60)print("\nExample 2: cos(x) = x in [0, π/2]")f2 = lambda x: np.cos(x) - xroot2 = ivt_root_finder(f2, 0, np.pi/2)print(f"\nThe equation cos(x) = x has solution x ≈ {root2:.6f}")print(f"Verification: cos({root2:.6f}) = {np.cos(root2):.6f}")

## 10. Application: Gradient Descent and Continuity**Why continuity matters in ML:**Gradient descent requires:1. **Continuous loss function** - no jumps or holes2. **Continuous gradients** (ideally) - smooth optimization landscape3. **Limits** - gradient is defined as a limit!$$\nabla L(\theta) = \lim_{h \to 0} \frac{L(\theta + h) - L(\theta)}{h}$$Let's visualize smooth vs non-smooth loss functions:

In [ ]:
# Smooth loss (MSE)def mse_loss(y_true, y_pred):    return np.mean((y_true - y_pred)**2)# Non-smooth loss (0-1 loss)def zero_one_loss(y_true, y_pred):    return np.mean(y_true != np.sign(y_pred))# Generate datanp.random.seed(42)y_true = np.array([1, -1, 1, -1, 1])theta_range = np.linspace(-3, 3, 300)# Compute losses for different predictionsmse_losses = []zo_losses = []for theta in theta_range:    y_pred = theta * np.ones_like(y_true)    mse_losses.append(mse_loss(y_true, y_pred))    zo_losses.append(zero_one_loss(y_true, y_pred))# Plotfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))# MSE (smooth)ax1.plot(theta_range, mse_losses, 'b-', linewidth=2)ax1.set_xlabel('θ (parameter)', fontsize=12)ax1.set_ylabel('Loss', fontsize=12)ax1.set_title('MSE Loss - Continuous & Smooth', fontsize=13, fontweight='bold')ax1.grid(True, alpha=0.3)ax1.text(0, max(mse_losses)*0.9, 'Gradient descent works!',          fontsize=11, ha='center', bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))# 0-1 Loss (discontinuous)ax2.plot(theta_range, zo_losses, 'r-', linewidth=2)ax2.set_xlabel('θ (parameter)', fontsize=12)ax2.set_ylabel('Loss', fontsize=12)ax2.set_title('0-1 Loss - Discontinuous', fontsize=13, fontweight='bold')ax2.grid(True, alpha=0.3)ax2.text(0, max(zo_losses)*0.9, 'Gradient descent fails!',          fontsize=11, ha='center', bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))plt.tight_layout()plt.show()print("Key Observations:")print("  • MSE is continuous and differentiable → gradients exist everywhere")print("  • 0-1 loss has jumps → no gradients at discontinuities")print("  • Modern ML uses smooth losses (MSE, cross-entropy) for optimization")print("\nActivation Functions:")print("  • Sigmoid, tanh: Smooth (continuous + differentiable)")print("  • ReLU: Continuous but not differentiable at 0")print("  • Step function: Discontinuous (rarely used)")

## 11. Practice ProblemsSolve these computationally and verify:**Problem 1:** Evaluate $\lim_{x \to 4} \frac{x^2 - 16}{x - 4}$**Problem 2:** Find one-sided limits of $f(x) = \begin{cases} x^2 - 1 & x < 2 \\ 3x & x \geq 2 \end{cases}$ at $x = 2$**Problem 3:** Evaluate $\lim_{x \to \infty} \frac{5x^2 + 2x - 1}{2x^2 - 3x + 4}$**Problem 4:** Find $k$ to make $f$ continuous: $f(x) = \begin{cases} \frac{x^2 + 3x - 10}{x - 2} & x \neq 2 \\ k & x = 2 \end{cases}$**Problem 5:** Use IVT to show $x^3 + x - 1 = 0$ has a solution in $(0, 1)$

In [ ]:
print("=" * 70)print("PRACTICE PROBLEM SOLUTIONS")print("=" * 70)# Problem 1print("\nProblem 1: lim(x→4) (x² - 16)/(x - 4)")f1 = lambda x: (x**2 - 16) / (x - 4)for x in [3.9, 3.99, 3.999, 4.001, 4.01, 4.1]:    print(f"  x = {x:.3f}: f(x) = {f1(x):.6f}")print("  Algebraic: (x² - 16)/(x - 4) = (x-4)(x+4)/(x-4) = x + 4")print("  Limit = 4 + 4 = 8")# Problem 2print("\n" + "=" * 70)print("\nProblem 2: One-sided limits at x=2")def f2(x):    return x**2 - 1 if x < 2 else 3*xleft_vals = [f2(2 - h) for h in [0.1, 0.01, 0.001]]right_vals = [f2(2 + h) for h in [0.1, 0.01, 0.001]]print(f"  Left limit: {left_vals[-1]}")print(f"  Right limit: {right_vals[-1]}")print(f"  Limit does NOT exist (3 ≠ 6)")# Problem 3print("\n" + "=" * 70)print("\nProblem 3: lim(x→∞) (5x² + 2x - 1)/(2x² - 3x + 4)")f3 = lambda x: (5*x**2 + 2*x - 1) / (2*x**2 - 3*x + 4)for x in [10, 100, 1000, 10000]:    print(f"  x = {x}: f(x) = {f3(x):.8f}")print("  Divide by x²: (5 + 2/x - 1/x²) / (2 - 3/x + 4/x²)")print("  As x → ∞: 5/2 = 2.5")# Problem 4print("\n" + "=" * 70)print("\nProblem 4: Find k for continuity")print("  f(x) = (x² + 3x - 10)/(x - 2) for x ≠ 2")print("  For continuity: k = lim(x→2) f(x)")f4 = lambda x: (x**2 + 3*x - 10) / (x - 2)# Factor: x² + 3x - 10 = (x+5)(x-2)print("  (x² + 3x - 10)/(x - 2) = (x+5)(x-2)/(x-2) = x + 5")k = 2 + 5print(f"  k = 2 + 5 = {k}")# Problem 5print("\n" + "=" * 70)print("\nProblem 5: IVT for x³ + x - 1 = 0 in (0, 1)")f5 = lambda x: x**3 + x - 1print(f"  f(0) = {f5(0)}")print(f"  f(1) = {f5(1)}")print(f"  f(0) < 0 < f(1), so by IVT, root exists")root = brentq(f5, 0, 1)print(f"  Root found: x ≈ {root:.8f}")print(f"  Verification: f({root:.6f}) = {f5(root):.2e}")

## 12. Self-Assessment ChecklistCheck your understanding:**Limits:**- [ ] Understand intuitive meaning of limits- [ ] Can compute limits using algebraic techniques- [ ] Know when to use factoring, rationalization, etc.- [ ] Understand ε-δ definition conceptually**One-Sided Limits:**- [ ] Can evaluate left and right limits separately- [ ] Know when two-sided limit exists vs doesn't exist- [ ] Can identify cases where one-sided limits differ**Infinite Limits:**- [ ] Understand notation $\lim_{x \to a} f(x) = \infty$- [ ] Can identify vertical asymptotes- [ ] Distinguish between limits at infinity and infinite limits**Limits at Infinity:**- [ ] Can evaluate $\lim_{x \to \infty} f(x)$ for rational functions- [ ] Know technique of dividing by highest power- [ ] Can identify horizontal asymptotes**Continuity:**- [ ] Know three conditions for continuity- [ ] Can test continuity at specific points- [ ] Understand continuous on an interval**Discontinuities:**- [ ] Can classify discontinuities (removable, jump, infinite, oscillating)- [ ] Know which discontinuities can be "fixed"- [ ] Understand how discontinuities affect function behavior**Theorems:**- [ ] Understand Intermediate Value Theorem statement- [ ] Can apply IVT to prove root existence- [ ] Understand Extreme Value Theorem implications**Applications:**- [ ] Know why continuity matters for gradient descent- [ ] Understand smooth vs non-smooth loss functions- [ ] Can connect limits to ML concepts (gradients, optimization)---## Next Steps**Week 10 Preview: Derivatives**- Formal definition using limits- Differentiation rules (power, product, quotient, chain)- Finding critical points- Applications to optimization- Relationship between derivatives and continuity---**Excellent work! Limits and continuity are the foundation of calculus. These concepts underpin gradient descent, neural network training, and virtually all continuous optimization in data science. You're now ready to tackle derivatives!**